# Set Membership

The cell below defines two **abstract classes**: the first represents a set and basic insert/search operations on it. You will need to impement this API four times, to implement (1) sequential search, (2) binary search tree, (3) balanced search tree, and (4) bloom filter. The second defines the synthetic data generator you will need to implement as part of your experimental framework. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [4]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

# abstract class to represent a set and its insert/search operations
class AbstractSet(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # inserts "element" in the set
    # returns "True" after successful insertion, "False" if the element is already in the set
    # element : str
    # inserted : bool
    @abstractmethod
    def insertElement(self, element):     
        inserted = False
        return inserted   
    
    # checks whether "element" is in the set
    # returns "True" if it is, "False" otherwise
    # element : str
    # found : bool
    @abstractmethod
    def searchElement(self, element):
        found = False
        return found    
    
    
    
# abstract class to represent a synthetic data generator
class AbstractTestDataGenerator(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # creates and returns a list of length "size" of strings
    # size : int
    # data : list<str>
    @abstractmethod
    def generateData(self, size):     
        data = [""]*size
        return data   


SequentialSearchSet Algorithm

In [5]:
class SequentialSearchSet:
    # Constructor initializes an empty list to store the data
    def __init__(self):
        self.data = []

    # Inserts an element into the data list if it is not already present
    def insertElement(self, element):
        if element not in self.data:
            self.data.append(element)
            return True
        return False

    # Searches for an element in the data list and returns True if found, False otherwise
    def searchElement(self, element):
        return element in self.data

    # Reads a file and inserts each word (space-separated) into the data list
    def read_file(self, file_name):
        with open(file_name, 'r') as f:
            for line in f:
                for word in line.split():
                    self.insertElement(word.strip())

    # Writes the contents of the data list to a file, one element per line
    def write_file(self, output_file_name):
        with open(output_file_name, 'w') as f:
            for element in self.data:
                f.write(element + '\n')


BinarySearchTreeSet Algorithm


In [6]:
class BinarySearchTreeSet(AbstractSet):
  
   class Node:
       def __init__(self, value):
           self.value = value
           self.left = None
           self.right = None
  
   def __init__(self):
       self.root = None

   #iterative implementations of search and insert element
   def insertElement(self, element):
       inserted = False
       if self.root is None:
           self.root = self.Node(element)
           inserted = True
      
       current = self.root
       inserted = False
       while True:
           if element == current.value:
               return inserted
           elif element < current.value:
               if current.left is None:
                   current.left = self.Node(element)
                   inserted = True
                   break
                  
               else:
                   current = current.left
           else:
               if current.right is None:
                   current.right = self.Node(element)
                   inserted = True
                   break
               else:
                   current = current.right
       return inserted
                  
   def searchElement(self, element):
       found = False
       current = self.root
       while current is not None:
           if element == current.value:
               found = True
               break
           elif element < current.value:
               current = current.left
           else:
               current = current.right
       return found

BalancedSearchTreeSet Algorithm

In [7]:
class BalancedSearchTreeSet(AbstractSet):
    # Define a nested Node class to store the keys and left/right children
    class Node:
        def __init__(self, key):
            self.key = key
            self.left = None
            self.right = None
            self.color = BalancedSearchTreeSet.RED  # Set the initial color to RED

    # Define constants for RED and BLACK colors
    RED = True
    BLACK = False

    # Return the number of elements in the set
    def __len__(self):
        return self.size

    # Initialize an empty set
    def __init__(self):
        self.root = None
        self.size = 0

    # Return an iterator that yields elements in sorted order
    def __iter__(self):
        def in_order_traversal(node):
            if node is not None:
                yield from in_order_traversal(node.left)
                yield node.key
                yield from in_order_traversal(node.right)

        return in_order_traversal(self.root)

    # Check if a node is RED
    def is_red(self, node):
        if node is None:
            return False
        return node.color == BalancedSearchTreeSet.RED

    # Rotate a node left
    def rotate_left(self, node):
        right_node = node.right
        node.right = right_node.left
        right_node.left = node
        right_node.color = node.color
        node.color = BalancedSearchTreeSet.RED

        return right_node

    # Rotate a node right
    def rotate_right(self, node):
        left_node = node.left
        node.left = left_node.right
        left_node.right = node
        left_node.color = node.color
        node.color = BalancedSearchTreeSet.RED

        return left_node

    # Flip the colors of a node and its left/right children
    def flip_colors(self, node):
        node.color = not node.color
        node.left.color = not node.left.color
        node.right.color = not node.right.color

        # Insert a new key into the set, maintaining balance
    def insertElement(self, element):
        # If the tree is empty, insert the new element as the root node and set its color to BLACK
        if not self.root:
            self.root = self.Node(element)
            self.root.color = BalancedSearchTreeSet.BLACK
            self.size += 1
            return

        # Initialize a stack to store the traversal path
        stack = []
        node = self.root

        # Traverse the tree until an empty spot is found or the key already exists
        while node is not None:
            if element == node.key:
                return  # The key already exists in the tree, do not insert
            stack.append(node)
            if element < node.key:
                if node.left is None:
                    node.left = self.Node(element)
                    break
                else:
                    node = node.left
            else:
                if node.right is None:
                    node.right = self.Node(element)
                    break
                else:
                    node = node.right

        # Now, balance the tree by traversing back up the stack
        while stack:
            parent = stack.pop()  # Get the next parent node from the stack

            # If the parent's right child is RED and the left child is BLACK or None, rotate the parent left
            if self.is_red(parent.right) and not self.is_red(parent.left):
                parent = self.rotate_left(parent)

            # If the parent's left child and its left child are both RED, rotate the parent right
            if parent.left and self.is_red(parent.left) and self.is_red(parent.left.left):
                parent = self.rotate_right(parent)

            # If the parent's left and right children are both RED, flip their colors and the parent's color
            if self.is_red(parent.left) and self.is_red(parent.right):
                self.flip_colors(parent)

            # Update the parent's parent to point to the balanced subtree
            if stack:
                if element < stack[-1].key:
                    stack[-1].left = parent
                else:
                    stack[-1].right = parent
            else:
                self.root = parent  # If there's no parent's parent, update the root

        # Ensure the root node is always colored BLACK
        self.root.color = BalancedSearchTreeSet.BLACK
        self.size += 1


    # Search for an element in the set and return True if found, otherwise False
    def searchElement(self, element):
        found = False
        node = self.root
        
        while node is not None and not found:
            if element == node.key:
                found = True
            elif element < node.key:
                node = node.left
            else:
                node = node.right
        
        return found

    # Read elements from a file and insert them into the set
    def read_file(self, file_name):
        with open(file_name, 'r') as f:
            for line in f:
                word = line.strip()
                self.insertElement(word)



BloomFilter Algorithm

In [8]:
import timeit

from bitarray import bitarray


class BloomFilter:
   def __init__(self):
       self.numHashes = 0
       self.bitArraySize = 0
       # false positives probability set to 1%
       self.falseProbability = 0.01
       # changing the word count affects bit array size and the number of hashes
       self.wordCount = 1000000

       self.setBitArraySize()
       self.bit_array = bitarray(self.bitArraySize)
       self.bit_array.setall(0)
       self.setNumHashes()

   @staticmethod
   def ln(x):
       n = 1000.0
       return n * ((x ** (1 / n)) - 1)

   # function calculates optimum bit array size for low false positive probability
   def setBitArraySize(self):
       self.bitArraySize = -(self.wordCount * (self.ln(self.falseProbability))) / ((self.ln(2)) ** 2)
       self.bitArraySize = int(self.bitArraySize)

   # optimum number of hashes ensure even distibution of hash values in bit array
   def setNumHashes(self):
       self.numHashes = (self.bitArraySize / self.wordCount) * self.ln(2)
       self.numHashes = int(self.numHashes)

   # takes the seed value 'num' to ensure each hash value is unique
   def _hash(self, element, num):
       hash_val = 0
       for char in str(element):
           hash_val = hash_val * num + ord(char)
       return hash_val % len(self.bit_array)

   # changes as many values of the bit array as there are hash functions
   def insertElement(self, element):
       for num in range(self.numHashes):
           index = self._hash(element, num)
           self.bit_array[index] = 1

   # computes the hash values and compares with bit array
   def searchElement(self, element):
       for num in range(self.numHashes):
           index = self._hash(element.lower(), num)
           if not self.bit_array[index]:
               return False
       return True


Testing insertElement and searchElement for SequntialSearchSet (Takes 30 minutes to run due to 10 rundowns for better approximation)

In [51]:
import timeit

# List of test files to process
test_files = ["test1-mobydick.txt", "test2-warpeace.txt", "test3-dickens.txt"]
search_file = "test-search.txt"

insertion_times = []
search_times = []
num_tests = 10

# Function to perform insertion operation and measure the time taken
def perform_insertion(file_name):
    seq_set = SequentialSearchSet()
    start_time = timeit.default_timer()
    seq_set.read_file(file_name)
    end_time = timeit.default_timer()
    seq_set.write_file(f"inserted-{file_name}")
    return seq_set, end_time - start_time

# Function to perform search operation and measure the time taken
def perform_search(seq_set, search_file):
    total_search_time = 0
    with open(search_file, 'r') as f:
        for line in f:
            word = line.strip()
            start_time = timeit.default_timer()
            found = seq_set.searchElement(word)
            end_time = timeit.default_timer()
            total_search_time += (end_time - start_time)
    return total_search_time

# Print header for the results table
print("Test File\t\tInsertion Time\t\tSearch Time")
print("-------------------------------------------------------")

# Process each test file for the specified number of times and print the average results
for test_file in test_files:
    print(f"Processing {test_file}")
    insertion_time_sum = 0
    search_time_sum = 0

    for i in range(num_tests):
        seq_set, insert_time = perform_insertion(test_file)
        search_time = perform_search(seq_set, search_file)
        insertion_time_sum += insert_time
        search_time_sum += search_time

    avg_insertion_time = insertion_time_sum / num_tests
    avg_search_time = search_time_sum / num_tests

    print(f"Insertion time: {avg_insertion_time:.6f} seconds")
    insertion_times.append(avg_insertion_time)
    
    print(f"Search time: {avg_search_time:.6f} seconds")
    search_times.append(avg_search_time)

    print()

# Print the summary of the results
print("Summary:")
print("-------------------------------------------------------")
for i in range(len(test_files)):
    print(f"{test_files[i]}\t\t{insertion_times[i]:.6f}\t\t{search_times[i]:.6f}")


Test File		Insertion Time		Search Time
-------------------------------------------------------
Processing test1-mobydick.txt
Insertion time: 3.163456 seconds
Search time: 0.017000 seconds

Processing test2-warpeace.txt
Insertion time: 6.246833 seconds
Search time: 0.015282 seconds

Processing test3-dickens.txt
Insertion time: 167.068107 seconds
Search time: 0.052943 seconds

Summary:
-------------------------------------------------------
test1-mobydick.txt		3.163456		0.017000
test2-warpeace.txt		6.246833		0.015282
test3-dickens.txt		167.068107		0.052943


Testing insertElement and searchElement for BinarySearchTree

In [50]:
import timeit

test_files = ["test1-mobydick.txt", "test2-warpeace.txt", "test3-dickens.txt"]
search_file = "test-search.txt"
num_tests = 10

# Function to perform insertion of elements from a test file into a BalancedSearchTreeSet
def perform_insertion(test_file):
    bst_set = BalancedSearchTreeSet()
    original_lines = []

    with open(test_file, 'r') as f:
        for line in f:
            original_lines.append(line)
            for word in line.split():
                bst_set.insertElement(word.strip())

    return bst_set, original_lines

# Function to write the inserted elements back to a new file
def write_inserted_file(file_name, lines):
    with open(file_name, 'w') as f:
        for line in lines:
            f.write(line)

# Function to perform search operation on the BalancedSearchTreeSet using words from a search file
def perform_search(bst_set, search_file):
    total_search_time = 0

    with open(search_file, 'r') as f:
        for line in f:
            for word in line.split():
                start_time = timeit.default_timer()
                _ = bst_set.searchElement(word.strip())
                end_time = timeit.default_timer()
                search_time = end_time - start_time
                total_search_time += search_time

    return total_search_time

# Print the header of the results table
print("Balanced Search Tree Set Results\n")
print("Test File\t\tInsertion Time\t\tSearch Time")
print("-------------------------------------------------------")

# Iterate through each test file, perform insertions and searches, and print the results of 10 different runs
for test_file in test_files:
    print(f"Processing {test_file}")
    
    insertion_times = []
    search_times = []
    for i in range(num_tests):
        start_time = timeit.default_timer()
        bst_set, original_lines = perform_insertion(test_file)
        end_time = timeit.default_timer()
        insertion_time = end_time - start_time
        insertion_times.append(insertion_time)

        write_inserted_file(f"inserted-{test_file}", original_lines)

        total_search_time = perform_search(bst_set, search_file)
        search_times.append(total_search_time)

    avg_insertion_time = sum(insertion_times) / num_tests
    avg_search_time = sum(search_times) / num_tests
    print(f"Insertion time: {avg_insertion_time:.6f} seconds")
    print(f"Search time: {avg_search_time:.6f} seconds")

print("-------------------------------------------------------")


Balanced Search Tree Set Results

Test File		Insertion Time		Search Time
-------------------------------------------------------
Processing test1-mobydick.txt
Insertion time: 0.371436 seconds
Search time: 0.000740 seconds
Processing test2-warpeace.txt
Insertion time: 0.726361 seconds
Search time: 0.000589 seconds
Processing test3-dickens.txt
Insertion time: 6.471281 seconds
Search time: 0.000972 seconds
-------------------------------------------------------


Testing insertElement and searchElement for BalancedSearchTree (1.24 minutes to run)

In [49]:
import timeit

test_files = ["test1-mobydick.txt", "test2-warpeace.txt", "test3-dickens.txt"]
search_file = "test-search.txt"
num_tests = 10

# Function to perform insertion of elements from a test file into a BalancedSearchTreeSet
def perform_insertion(test_file):
    bst_set = BinarySearchTreeSet()
    original_lines = []

    with open(test_file, 'r') as f:
        for line in f:
            original_lines.append(line)
            for word in line.split():
                bst_set.insertElement(word.strip())

    return bst_set, original_lines

# Function to write the inserted elements back to a new file
def write_inserted_file(file_name, lines):
    with open(file_name, 'w') as f:
        for line in lines:
            f.write(line)

# Function to perform search operation on the BalancedSearchTreeSet using words from a search file
def perform_search(bst_set, search_file):
    total_search_time = 0

    with open(search_file, 'r') as f:
        for line in f:
            for word in line.split():
                start_time = timeit.default_timer()
                _ = bst_set.searchElement(word.strip())
                end_time = timeit.default_timer()
                search_time = end_time - start_time
                total_search_time += search_time

    return total_search_time

# Print the header of the results table
print("Binary Search Tree Set Results\n")
print("Test File\t\tInsertion Time\t\tSearch Time")
print("-------------------------------------------------------")

# Iterate through each test file, perform insertions and searches, and print the results of 10 different runs
for test_file in test_files:
    print(f"Processing {test_file}")
    
    insertion_times = []
    search_times = []
    for i in range(num_tests):
        start_time = timeit.default_timer()
        bst_set, original_lines = perform_insertion(test_file)
        end_time = timeit.default_timer()
        insertion_time = end_time - start_time
        insertion_times.append(insertion_time)

        write_inserted_file(f"inserted-{test_file}", original_lines)

        total_search_time = perform_search(bst_set, search_file)
        search_times.append(total_search_time)

    avg_insertion_time = sum(insertion_times) / num_tests
    avg_search_time = sum(search_times) / num_tests
    print(f"Insertion time: {avg_insertion_time:.6f} seconds")
    print(f"Search time: {avg_search_time:.6f} seconds")

print("-------------------------------------------------------")


Binary Search Tree Set Results

Test File		Insertion Time		Search Time
-------------------------------------------------------
Processing test1-mobydick.txt
Insertion time: 0.193052 seconds
Search time: 0.000662 seconds
Processing test2-warpeace.txt
Insertion time: 0.514529 seconds
Search time: 0.000720 seconds
Processing test3-dickens.txt
Insertion time: 4.407513 seconds
Search time: 0.000830 seconds
-------------------------------------------------------


Testing insertElement and searchElement for BloomFilter

In [48]:
import timeit
test_files = ["test1-mobydick.txt", "test2-warpeace.txt", "test3-dickens.txt"]
search_file = "test-search.txt"


# Function to perform insertion of elements from a test file into a BloomFilterSet
def perform_insertion(test_file):
   bloom_set = BloomFilter()
   original_lines = []

   with open(test_file, 'r') as f:
       for line in f:
           original_lines.append(line)
           for word in line.split():
               bloom_set.insertElement(word.strip())

   return bloom_set, original_lines


# Function to write the inserted elements back to a new file
def write_inserted_file(file_name, lines):
   with open(file_name, 'w') as f:
       for line in lines:
           f.write(line)


# Function to perform search operation on the BloomFilterSet using words from a search file
def perform_search(bloom_set, search_file):
   total_search_time = 0

   with open(search_file, 'r') as f:
       for line in f:
           for word in line.split():
               start_time = timeit.default_timer()
               _ = bloom_set.searchElement(word.strip())
               end_time = timeit.default_timer()
               search_time = end_time - start_time
               total_search_time += search_time

   return total_search_time


# Print the header of the results table
print("Bloom Filter Set Results\n")
print("Test File\t\tInsertion Time\t\tSearch Time")
print("-------------------------------------------------------")

# Iterate through each test file, perform insertions and searches, and print the results
for test_file in test_files:
   print(f"Processing {test_file}")

   start_time = timeit.default_timer()
   bloom_set, original_lines = perform_insertion(test_file)
   end_time = timeit.default_timer()
   insertion_time = end_time - start_time
   print(f"Insertion time: {insertion_time:.6f} seconds")

   write_inserted_file(f"inserted-{test_file}", original_lines)

   total_search_time = perform_search(bloom_set, search_file)
   print(f"Search time: {total_search_time:.6f} seconds")

print("-------------------------------------------------------")


Bloom Filter Set Results

Test File		Insertion Time		Search Time
-------------------------------------------------------
Processing test1-mobydick.txt
Insertion time: 0.459903 seconds
Search time: 0.001518 seconds
Processing test2-warpeace.txt
Insertion time: 1.236451 seconds
Search time: 0.001361 seconds
Processing test3-dickens.txt
Insertion time: 10.894398 seconds
Search time: 0.001444 seconds
-------------------------------------------------------


In [2]:
import string
import random


class SyntheticDataGenerator:
  
   def __init__(self, minLength=1, maxLength=10):
       self.minLength = minLength
       self.maxLength = maxLength
              
   def generateData(self, size):
       data = ["".join(random.choice(string.ascii_lowercase) for _ in range(self.maxLength))
               for _ in range(size)]
      
       return data

In [3]:
import string
import random


class SyntheticSortedDataGenerator:
  
   def __init__(self, minLength=1, maxLength=10):
       self.minLength = minLength
       self.maxLength = maxLength
              
   def generateData(self, size):
       # Generate a list of strings then sort in reverse order
       data = ["".join(random.choice(string.ascii_lowercase) for _ in range(self.maxLength))
               for _ in range(size)]
       data.sort(reverse=True)
      
       return data

In [13]:
#code to evaluate insertion time of 15000 synthetic elements into data structures- lists used for graph plotting

import timeit

data_generator = SyntheticDataGenerator(1, 5)
words = data_generator.generateData(15000)

inserted1 = []
time1 = []
cumulative_time1 = 0
bst = BinarySearchTreeSet()
for j in range(1000, len(words), 1000):
    statement_code = f"""
for word in words[{j - 1000}:{j}]:
    bst.insertElement(word.lower())
"""

    runtime = timeit.timeit(stmt=statement_code, globals=globals(), number=10)
    cumulative_time1 += runtime/10

    inserted1.append(j)
    time1.append(runtime / 10)

inserted2 = []
time2 = []
cumulative_time2 = 0
rb = BalancedSearchTreeSet()
for j in range(1000, len(words), 1000):
    statement_code = f"""
for word in words[{j - 1000}:{j}]:
    rb.insertElement(word.lower())
"""

    runtime = timeit.timeit(stmt=statement_code, globals=globals(), number=10)
    cumulative_time2 += runtime/10

    inserted2.append(j)
    time2.append(runtime / 10)

inserted3 = []
time3 = []
cumulative_time3 = 0
bf = BloomFilter()
for j in range(1000, len(words), 1000):
    statement_code = f"""
for word in words[{j - 1000}:{j}]:
    bf.insertElement(word.lower())
"""

    runtime = timeit.timeit(stmt=statement_code, globals=globals(), number=10)
    cumulative_time3 += runtime/10

    inserted3.append(j)
    time3.append(runtime / 10)

inserted4 = []
time4 = []
cumulative_time4 = 0
ss = SequentialSearchSet()
for j in range(1000, len(words), 1000):
    statement_code = f"""
for word in words[{j - 1000}:{j}]:
    ss.insertElement(word.lower())
"""

    runtime = timeit.timeit(stmt=statement_code, globals=globals(), number=10)
    cumulative_time4 += runtime/10

    inserted4.append(j)
    time4.append(runtime / 10)

print("\nCumulative insertion time for Binary Search Tree: {:.6f} seconds".format(cumulative_time1))
print("Cumulative insertion time for Balanced Search Tree: {:.6f} seconds".format(cumulative_time2))
print("Cumulative insertion time for Bloom Filter: {:.6f} seconds".format(cumulative_time3))
print("Cumulative insertion time for Sequential Search: {:.6f} seconds".format(cumulative_time4))



Cumulative insertion time for Binary Search Tree: 0.016963 seconds
Cumulative insertion time for Balanced Search Tree: 0.025220 seconds
Cumulative insertion time for Bloom Filter: 0.030192 seconds
Cumulative insertion time for Sequential Search: 0.689559 seconds


In [12]:
#code to evaluate edge case of insertion of sorted lists- arrays again maintained for graph plotting
import timeit

dataGenerator = SyntheticDataGenerator(1, 10)
words = dataGenerator.generateData(10000)

skewedDataGenerator = SyntheticSortedDataGenerator(1, 10)
skewedWords = skewedDataGenerator.generateData(10000)

inserted1 = []
time1 = []
cumulative_time1 = 0

# Create a BinarySearchTreeSet object outside the loop
bst = BalancedSearchTreeSet()
for j in range(0, len(words), 1):
    statement_code = f"""
for word in words[{j - 1}:{j}]:
    bst.insertElement(word.lower())
"""

    # Time the execution of the insertElement() method
    runtime = timeit.timeit(stmt=statement_code, globals=globals(), number=1000)
    cumulative_time1 += runtime/1000

    inserted1.append(j)
    time1.append(runtime/1000)

inserted2 = []
time2 = []
cumulative_time2 = 0

# Create a BinarySearchTreeSet object outside the loop
bst = BalancedSearchTreeSet()
for j in range(0, len(skewedWords), 1):
    statement_code = f"""
for word in skewedWords[{j - 1}:{j}]:
    bst.insertElement(word.lower())
"""

    # Time the execution of the insertElement() method
    runtime = timeit.timeit(stmt=statement_code, globals=globals(), number=1000)
    cumulative_time2 += runtime/1000

    inserted2.append(j)
    time2.append(runtime/1000)

print("\nCumulative time for randomly generated data: {:.6f} seconds, total elements inserted: {}".format(cumulative_time1, len(inserted1)))
print("Cumulative time skewed data: {:.6f} seconds, total elements inserted: {}".format(cumulative_time2, len(inserted2)))



Cumulative time for randomly generated data: 0.010537 seconds, total elements inserted: 10000
Cumulative time skewed data: 0.014122 seconds, total elements inserted: 10000


Code below contains functions used to write report:

In [45]:

def sortingEval(filenames):
    sortedPercentages = {}

    for filename in filenames:
        with open(filename, 'r') as file:
            text = file.read()
            words = text.split()
            wordPairs = zip(words[:-1], words[1:]) #every possible word pair in file
            sortedPairs = sum(1 for a, b in wordPairs if a <= b) #add to count if pair is sorted
            totalPairs = len(words) - 1 #find total number of pairs
            sortedPercentage = (sortedPairs / totalPairs) * 100
            sortedPercentages[filename] = sortedPercentage

    return sortedPercentages

filenames = ["test1-mobydick.txt", "test2-warpeace.txt", "test3-dickens.txt"]
sortedPercentages = sortingEval(filenames)

for filename, sortedPercentage in sortedPercentages.items():
    print(f"The text file '{filename}' is {sortedPercentage:.2f}% sorted.")


The text file 'test1-mobydick.txt' is 49.12% sorted.
The text file 'test2-warpeace.txt' is 48.73% sorted.
The text file 'test3-dickens.txt' is 48.72% sorted.


In [46]:
bst = BinarySearchTreeSet()
# Used this code to find the height of 3 BSTs- note as it stands this code finds height of trees produced by whole files- 
# we reduced size to 100,000 elements before using as referenced in report

def calcTreeHeights(filenames, bst):
    treeHeights = {}

    #insert elements from each file into bst structure
    for filename in filenames:
        with open(filename, 'r') as file:
            elements = []
            for line in file:
                elements.extend(line.strip().split())
            for word in elements:
                bst.insertElement(word)

        # Function to calculate the height of the tree recursively
        def height(node):
            if not node:
                return 0
            return 1 + max(height(node.left), height(node.right))

        # store tree height in tree_heights dictionary
        treeHeights[filename] = height(bst.root)

        # Reset the BST for the next file
        bst.root = None
        
    return treeHeights

filenames = ["test1-mobydick.txt", "test2-warpeace.txt", "test3-dickens.txt"]
treeHeights = calcTreeHeights(filenames, bst)

for filename, height in treeHeights.items():
    print(f"The height of the tree for the file '{filename}' is {height}.")


The height of the tree for the file 'test1-mobydick.txt' is 35.
The height of the tree for the file 'test2-warpeace.txt' is 37.
The height of the tree for the file 'test3-dickens.txt' is 41.


In [47]:
def count_words(fileNames):
    with open(fileNames, 'r') as file:
        text = file.read()
        words = text.split()
        return len(words)

fileNames = ["test1-mobydick.txt", "test2-warpeace.txt", "test3-dickens.txt"] 

for index, file_path in enumerate(fileNames, start=0):
    numWords = count_words(file_path)
    print(f'The number of words in file {fileNames[index]} is: {numWords}')

The number of words in file test1-mobydick.txt is: 209329
The number of words in file test2-warpeace.txt is: 564236
The number of words in file test3-dickens.txt is: 5149661
